In [37]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, BatchNormalization, Activation, MaxPooling1D, Flatten, Dense, Dropout


In [38]:
X = pd.read_csv(r'E:\4th sem\IOBS-2\project\Non Coding RNA classification\Cleaned_data\EIIP\0-100_features.csv')
y = pd.read_csv(r"E:\4th sem\IOBS-2\project\Non Coding RNA classification\Cleaned_data\EIIP\0-100_class.csv")

In [68]:
# Define model parameters
filters = 64
kernel_size = 5
growth_rate = 32
num_layers_per_block = 3
pool_size = 2
dropout_rate = 0.3

# Define the model architecture
inputs = Input(shape=(100, 1))

In [40]:
from sklearn.model_selection import train_test_split

# Split data into train (80%) and combined test/validation (20%)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.1, shuffle=True, random_state=42)

# Split combined test/validation into test (10%) and validation (10%)
X_test, X_val, y_test, y_val = train_test_split(X_temp, y_temp, test_size=0.50, shuffle=True, random_state=42)

# Display the shapes of the resulting sets
print("Train set shapes:", X_train.shape, y_train.shape)
print("Test set shapes:", X_test.shape, y_test.shape)
print("Validation set shapes:", X_val.shape, y_val.shape)


Train set shapes: (1306, 100) (1306, 1)
Test set shapes: (73, 100) (73, 1)
Validation set shapes: (73, 100) (73, 1)


In [41]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Fit LabelEncoder and transform labels to integers
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)
y_val_encoded = label_encoder.transform(y_val)

# Convert integer labels to one-hot encoded labels
y_train = to_categorical(y_train_encoded)
y_test = to_categorical(y_test_encoded)
y_val = to_categorical(y_val_encoded)


c:\Users\neelr\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\neelr\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\preprocessing\_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
c:\Users\neelr\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\preprocessing\_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


In [72]:
from tensorflow.keras.layers import Concatenate
# Initial convolutional layer
x = Conv1D(filters, kernel_size=kernel_size, padding='same')(inputs)
x = BatchNormalization()(x)
x = Activation('relu')(x)

# Dense blocks
def dense_block(x, growth_rate, num_layers):
    for _ in range(num_layers):
        # Bottleneck layer
        bottleneck = BatchNormalization()(x)
        bottleneck = Activation('relu')(bottleneck)
        bottleneck = Conv1D(filters=growth_rate, kernel_size=1, padding='same')(bottleneck)

        # Composite function: BN + ReLU + 3x3 Conv
        x = BatchNormalization()(bottleneck)
        x = Activation('relu')(x)
        x = Conv1D(filters=growth_rate, kernel_size=3, padding='same')(x)

        # Concatenate the output of the composite function with the input to the block
        x = Concatenate(axis=-1)([x, bottleneck])
    return x

# x = dense_block(x, growth_rate, num_layers_per_block)
# x = dense_block(x%10, growth_rate, num_layers_per_block)

# Additional convolutional layer
# x = Conv1D(filters * 2, kernel_size=3, padding='same')(x)  # Double filters after dense blocks
# x = BatchNormalization()(x)
# x = Activation('relu')(x)


# Additional convolutional layer
x = Conv1D(filters * 3 , kernel_size=3, padding='same')(x)  # Double filters after dense blocks
x = BatchNormalization()(x)
x = Activation('relu')(x)


# Additional layers
x = MaxPooling1D(pool_size=pool_size)(x)
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(dropout_rate)(x)
x = Dense(32, activation='relu')(x)
x = Dropout(dropout_rate)(x)

# Output layer
outputs = Dense(12, activation='softmax')(x)  # Assuming you have 12 classes

# Create the model
model = Model(inputs=inputs, outputs=outputs)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print model summary
model.summary()

Model: "model_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 100, 1)]          0         
                                                                 
 conv1d_130 (Conv1D)         (None, 100, 64)           384       
                                                                 
 batch_normalization_133 (B  (None, 100, 64)           256       
 atchNormalization)                                              
                                                                 
 activation_133 (Activation  (None, 100, 64)           0         
 )                                                               
                                                                 
 conv1d_131 (Conv1D)         (None, 100, 192)          37056     
                                                                 
 batch_normalization_134 (B  (None, 100, 192)          768

In [73]:
# Fit the model
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_val, y_val))

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test)

Epoch 1/20
41/41 [==============================] - 2s 18ms/step - loss: 2.5958 - accuracy: 0.3476 - val_loss: 2.4402 - val_accuracy: 0.2740
Epoch 2/20
41/41 [==============================] - 1s 14ms/step - loss: 1.3643 - accuracy: 0.5513 - val_loss: 2.4229 - val_accuracy: 0.0685
Epoch 3/20
41/41 [==============================] - 1s 15ms/step - loss: 1.1725 - accuracy: 0.5995 - val_loss: 2.4473 - val_accuracy: 0.0685
Epoch 4/20
41/41 [==============================] - 1s 15ms/step - loss: 1.0160 - accuracy: 0.6593 - val_loss: 2.6068 - val_accuracy: 0.0685
Epoch 5/20
41/41 [==============================] - 1s 14ms/step - loss: 0.9827 - accuracy: 0.6623 - val_loss: 2.6078 - val_accuracy: 0.1370
Epoch 6/20
41/41 [==============================] - 1s 15ms/step - loss: 0.8945 - accuracy: 0.7037 - val_loss: 2.7489 - val_accuracy: 0.0685
Epoch 7/20
41/41 [==============================] - 1s 16ms/step - loss: 0.8680 - accuracy: 0.6945 - val_loss: 2.6930 - val_accuracy: 0.1507
Epoch 8/20
41